# SPI Pipeline

1. Get info about users in Instagram and Twitter
2. Compute sentiment
3. Get percentiles


## Import libraries

In [1]:
from instagrapi                  import Client
from datetime                    import date
from transformers                import AutoTokenizer, AutoModelForSequenceClassification
from pysentimiento.preprocessing import preprocess_tweet
from pysentimiento               import create_analyzer
from datetime                    import datetime as dt
from pandas.api.types            import is_numeric_dtype
from scipy                       import stats
import pandas   as  pd
import numpy    as  np

import torch
import tweepy
import time

c:\Users\Usuario\Desktop\ITSENSE\vertikal-x-ds\vertikal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import models

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("papluca/xlm-roberta-base-language-detection")
language_detector = AutoModelForSequenceClassification.from_pretrained("papluca/xlm-roberta-base-language-detection")
analyzer_en = create_analyzer(task="sentiment", lang="en")
analyzer_es = create_analyzer(task="sentiment", lang="es")

In [13]:
def rescale_probs(proba_dict):
    keys, values = list(proba_dict.keys()), list(proba_dict.values())
    pred_key = keys[np.argmax(values)]
    pred_value = 0
    if pred_key == "NEG":
        pred_value = 1-np.max(values)
    elif pred_key == "POS":
        pred_value = np.max(values)
    else:
        neg_value = values[0]
        pos_value = values[-1]

        add_val = pos_value if pos_value > neg_value else -neg_value

        pred_value = 0.5 + (1-np.max(values))*add_val/2
    return pred_value
    
def preprocess_comment_adv(string_, max_word_count=50):
    substrs_to_remove = ["cara emoji", "emoji", "   ", "\n"]
    procs_str = preprocess_tweet(string_)
    for substr in substrs_to_remove:
        procs_str = procs_str.replace(substr, "")
    procs_str = procs_str.replace("Jjaja", "Jajaja")
    
    return " ".join(procs_str.split(" ")[:max_word_count])

def get_sentiment(comments, lang_detector, en_analyzer, sp_analyzer, tokenizer):
    # Quitar esto
    if len(comments) == 0: # Sentimiento del usuario cuando no tiene comentarios = 0.5. Debería ser menor a 0.5 (e.g. 0.2)
        return 0.2
    else:
        preprocessed_comments = [preprocess_comment_adv(comment) for comment in comments]
        languages_detected = []
        for comment in preprocessed_comments:
            inputs = tokenizer(comment, return_tensors="pt")
            with torch.no_grad():
                logits = lang_detector(**inputs).logits
            predicted_class_id = logits.argmax().item()
            languages_detected.append(lang_detector.config.id2label[predicted_class_id])

        sentiment_probas = []
        for ix in range(len(preprocessed_comments)):
            if languages_detected[ix] == "en":
                estimation = en_analyzer.predict(preprocessed_comments[ix]).probas
            elif languages_detected[ix] == "es" or languages_detected[ix] == "pt":
                estimation = sp_analyzer.predict(preprocessed_comments[ix]).probas
            else:
                estimation = {'NEG': 0, 'NEU': 1, 'POS': 0}
            sentiment_probas.append(estimation)

        sentiment = [rescale_probs(probs) for probs in sentiment_probas]
        df = pd.DataFrame({"Comment": comments, "Language": languages_detected, "Sentiment": sentiment})

        return df["Sentiment"].mean()

## Instantiate tokens and declare methods for Instagram and Twitter retrieval of data

In [38]:
# --- INSTAGRAM ---

cl = Client()
# cl.login("clyde_donovan_2022", "clyde_donovan_2022_")
cl.login("chrismiller202212", "1OE0t5@Po9*z_")
# cl.login("samanthaoakley202212", "1OE0t5@Po9*z")

def extract_text(comment):
    comments_text_ls = []
    if len(comment[0]) != 0:
        comments_text_ls = [comment.text for comment in comment[0]]
    return comments_text_ls

def get_ig_data(cl, username, max_posts=10, max_comments=10, retrieve_all=False): 

    user_id = cl.user_id_from_username(username)
    if retrieve_all:
        posts = cl.user_medias(user_id)
    else: 
        posts = cl.user_medias(user_id, max_posts)
    user_information = cl.user_info(user_id)

    n_followers = user_information.follower_count
    n_following = user_information.following_count
    n_posts = user_information.media_count

    # time.sleep(60)
    
    posts_info = {}
    
    for i, post in enumerate(posts):
        if i >= max_posts:
            posts_info[post.id] = {
                                "n_comments": post.comment_count, 
                                "n_likes": post.like_count, 
                                "caption": post.caption_text, 
                                "comments_text": []
                                }    
        else: 
            posts_info[post.id] = {
                                    "n_comments": post.comment_count, 
                                    "n_likes": post.like_count, 
                                    "caption": post.caption_text, 
                                    "comments_text": extract_text(cl.media_comments_chunk(post.id, max_amount=max_comments))
                                    }                              
    # time.sleep(60)
    user_data = {
        "username": username,
        "user_id": user_id,
        "n_followers": n_followers,
        "n_following": n_following,
        "n_posts_total": n_posts,
        "n_posts_retrieved": len(posts_info),
        "n_likes_total": sum([posts_info[key]["n_likes"] for key in posts_info.keys()]),
        "n_likes_retrieved": sum([posts_info[key]["n_likes"] for key in list(posts_info.keys())[:max_posts]]),
        "n_comments_total": sum([posts_info[key]["n_comments"] for key in posts_info.keys()]),
        "n_comments_retrieved": sum([len(posts_info[key]["comments_text"]) for key in posts_info.keys()]),
        "created_at": dt.now(),
        "posts_info": posts_info
    }
    return user_data

# --- TWITTER ---

CONSUMER_KEY = 'P8uR3oU2eTF1hZw2SX1lma8Zw'
CONSUMER_SECRET = 'Kg85hTJdliuTdFroydQvoRsg7cpr5WZm2MdEQtCz8EjcjG04dp'
ACCESS_TOKEN = '386339280-5sy1Smvnkw91cBObIwu3ju6aMQJU4B8X8HZNsDSo'
ACCESS_TOKEN_SECRET = 'WuZNLBRfzZgT6DIDnZOPqaVlt8uMtrvOUJ9aSbUwkmocD'

auth = tweepy.OAuth1UserHandler(
    CONSUMER_KEY, 
    CONSUMER_SECRET, 
    ACCESS_TOKEN, 
    ACCESS_TOKEN_SECRET
)

api = tweepy.API(auth, wait_on_rate_limit=True)

twitter_username = ""

def get_replies(api, username, tweet_id, max_replies=10, max_attempts=20):
    replies = tweepy.Cursor(api.search_tweets, q='to:{}'.format(username),
                                    since_id=tweet_id, tweet_mode='extended').items()

    replies_ls = []

    counter_fetched_rep = 0
    counter_attempts = 0
    while counter_fetched_rep < max_replies and counter_attempts < max_attempts:
        try:
            reply = replies.next()
            if not hasattr(reply, 'in_reply_to_status_id_str'):
                continue
            if reply.in_reply_to_status_id == tweet_id:
                replies_ls.append(reply.full_text)
                counter_fetched_rep = counter_fetched_rep + 1 
            counter_attempts = counter_attempts + 1 

        except StopIteration:
            break

        except Exception as e:
            print("Failed while fetching replies {}".format(e))
            break
    return replies_ls

def get_twitter_data(api, username, max_tweets=5, max_replies=10):
    
    n_followers = api.search_users(username)[0].followers_count
    extracted_tweets = []

    for status in tweepy.Cursor(api.search_tweets, 
                                f"from:{username}",
                                count=max_tweets).items(max_tweets):
        extracted_tweets.append(status)
    retweets = [tweet for tweet in extracted_tweets if "RT @" in tweet.text]
    tweets = set(extracted_tweets) - set(retweets)
    n_retweets = len(retweets)
    n_tweets = len(tweets)
    n_id_ls = []
    n_retweets_ls = []
    n_favorites_ls = []
    tweet_text_ls = []
    for tweet in tweets:
        n_id_ls.append(tweet.id)
        n_retweets_ls.append(tweet.retweet_count)
        n_favorites_ls.append(tweet.favorite_count)
        tweet_text_ls.append(tweet.text)
    replies_ls = [get_replies(api, username, tweet_id, max_replies=max_replies) for tweet_id in n_id_ls]

    user_data = {
        "user_name": username,
        "n_followers": n_followers,
        "n_retweets": n_retweets,
        "n_tweets": n_tweets,
        "n_retweets_to_user": sum(n_retweets_ls),
        "n_favorites_to_user": sum(n_favorites_ls),
        "n_replies_to_user": sum([len(replies_post) for replies_post in replies_ls]),
        "tweets_text": tweet_text_ls,
        "tweets_replies": replies_ls,
        "created_at": dt.now()
    }

    return user_data

def get_comments_ls(data_dict, mode="ig"):
    all_comments = []
    if mode=="ig":
        for post_id in data_dict['posts_info'].keys():
            all_comments.extend(data_dict['posts_info'][post_id]["comments_text"])
    else:
        for tweet_replies in data_dict["tweets_replies"]:
            for comment in tweet_replies:
                all_comments.append(comment)
    return all_comments

## Retrieve data from social media

In [3]:
user_data_twitter = get_twitter_data(api, "JFCadavid", max_tweets=10, max_replies=10)

In [4]:
user_data_ig = get_ig_data(cl, "jfcadavid", max_posts=5, max_comments=10)

In [5]:
def get_comments_ls(data_dict, mode="ig"):
    all_comments = []
    if mode=="ig":
        for post_id in data_dict['posts_info'].keys():
            all_comments.extend(data_dict['posts_info'][post_id]["comments_text"])
    else:
        for tweet_replies in data_dict["tweets_replies"]:
            for comment in tweet_replies:
                all_comments.append(comment)
    return all_comments

## Compute the sentiment for user accounts (IG and Twitter) 

In [14]:
# Instagram
# posts_comments_ig = [user_data_ig['posts_info'][post_id]['comments_text'] for post_id in user_data_ig['posts_info'].keys()]
posts_comments_ig = get_comments_ls(user_data_ig, mode="ig")
sentiment_instagram = get_sentiment(posts_comments_ig, language_detector, analyzer_en, analyzer_es, tokenizer)

In [15]:
# Twitter
posts_comments_tw = get_comments_ls(user_data_twitter, mode="tw")
sentiment_twitter = get_sentiment(posts_comments_ig, language_detector, analyzer_en, analyzer_es, tokenizer)

In [9]:
posts_comments_ig

['@juancarlos.velazquezcorrea solo tiene el derecho de trasmisión de 32 partidos, la exclusiva la tiene DirecTV',
 'Pero ya ni los partidos buenos dan 🤦\u200d♂️, definitivamente caracol y rcn de mal en peor solo repetideras y realitys repetitivos, canales qué a futuro se extinguirán si siguen así.',
 'Por favor mantengan las buenas obras, estaba emocionado cuando abrí mi billetera y vi la ganancia de 8500 € de la inversión de 500 €. @jannike_fx_expert📩',
 'Lastima que ese regalo no lo pudimos disfrutar muchos por qué los canales ninguno lo transmitió, pero si fuera un partido bien regular le hacen pre pos y comentarios',
 'Gracias mi Señor  Grace, de hecho usted es una gran mujer para llamarse nunca espero esto de usted señora le demuestro que aclararé más por lo que ha hecho en mi vida señora prometo que el mundo sepa lo que tiene  hecho en mi vida hoy sal seguiré compartiendo las buenas noticias mrs.  @grace__crypto_fx',
 'Una pregunta porque no transmitieron el juego de Alemania vs 

In [8]:
posts_comments_tw

['@JFCadavid @AS_Colombia Sin desconocer la calidad de modric, Iniesta! Siempre Iniesta! Es un genio! Y soy madridista fervoroso. \n Pero "al cesar...."',
 '@JFCadavid @cmforex1 @Jwillocv @GolCaracol @CaracolRadio No le puse el "(?)"',
 '@JFCadavid @sebabotero11 No paso nada',
 '@JFCadavid @sebabotero11 Sebas que pena, quien es el moreno con la NRO 6. Y podían jugar con antioquia así fueran se otras regiones ? César es de Choco y Aldo Samario.',
 '@JFCadavid Abrazo grande Juan !',
 '@JFCadavid Emocion plena papa!!!!']

In [52]:
sentiment_twitter

[0.41033692056399884,
 0.5,
 0.3339894809149695,
 0.27873771263383934,
 0.27252542562257887]

In [53]:
print(f"Instagram sentiment: {np.mean(sentiment_instagram)}")
print(f"Twitter sentiment: {np.mean(sentiment_twitter)}")

Instagram sentiment: 0.4369447707948009
Twitter sentiment: 0.3591179079470773


In [54]:
user_data_ig

{'user_name': 'jfcadavid',
 'user_id': '192185562',
 'n_followers': 130317,
 'n_following': 756,
 'n_posts_total': 2834,
 'n_posts_retrieved': 5,
 'n_likes_total': 5049,
 'n_comments_total': 70,
 'date': '21/11/2022',
 'posts_info': {'2975297232093322943_192185562': {'n_comments': 56,
   'n_likes': 1230,
   'caption': '💔 ¿quién más así?',
   'comments_text': ['Yo 😭',
    '😢',
    '🙋🏽\u200d♂️ 😥',
    '😢😢😢😢',
    '☝🏻',
    '😢',
    '\U0001fae4🙋🏼\u200d♀️',
    'Mala noticia el mejor del mundo',
    '💔',
    'Valbuena debe estar igual y las chicas que conocieron con Roberi también... es un crack, es un animal en la cancha, pero la vida se encarga de cobrarte todo, algunos lo llaman Karma 🙌',
    '😢😢😢',
    '☝️',
    '@john_alex86 si',
    '😢🙋🏼\u200d♀️😢',
    'BENZEMA ES MAS SALADO QUE ZIPAQUIRA.  UN NGRAN GOLEADOR SE VA DEL MUNDIAL.  COSAS DE LA VIDA.  Y COLOMBIA EXTRAÑANDO A  LOS PECHIFRIOS QUE DE PREMIO POR SER ELIMINADOS LOS LLEVAN DE PASEO A ESTADOS.  COSAS DEL FÚTBOL',
    'Probableme

## Create a dummy dict with account users

In [40]:
users_dict = {
    "cristiano": {
        "instagram": "cristiano",
        "twitter": "Cristiano"
    },
    "messi": {
        "instagram": "leomessi",
        "twitter": "Ieomessiok"
    },
    "lebron": {
        "instagram": "lebron",
        "twitter": "KingJames"
    },
    "nadal": {
        "instagram": "rafaelnadal",
        "twitter": "RafaelNadal"
    },
    "mayweather": {
        "instagram": "floydmayweather",
        "twitter": "FloydMayweather"
    }
}

## Create a gigamethod to process everything

In [32]:
def get_spi(user_dict, tw_api, ig_client, retrieve_all_ig=False):
# Twitter

    start_time = time.time()
    print("Getting twitter profile data")
    twitter_username = user_dict["twitter"]
    user_data_twitter = get_twitter_data(tw_api, twitter_username, max_tweets=10, max_replies=10)

    posts_comments_tw = get_comments_ls(user_data_twitter, mode="tw")
    sentiment_twitter = get_sentiment(posts_comments_tw, language_detector, analyzer_en, analyzer_es, tokenizer)
    user_data_twitter.pop('tweets_text', None)
    user_data_twitter.pop('tweets_replies', None)
    user_data_twitter["sentiment_twitter"] = sentiment_twitter
    print(f"Elapsed time: {(time.time() - start_time)} secs")
    
    
# Instagram
    start_time = time.time()
    print("Getting instagram profile data")
    ig_username = user_dict["instagram"]
    user_data_ig = get_ig_data(ig_client,ig_username, max_posts=5, max_comments=10, retrieve_all=retrieve_all_ig)

    posts_comments_ig = get_comments_ls(user_data_ig, mode="ig")
    sentiment_instagram = get_sentiment(posts_comments_ig, language_detector, analyzer_en, analyzer_es, tokenizer)
    user_data_ig.pop('posts_info', None)
    user_data_ig["sentiment_instagram"] = sentiment_instagram
    print(f"Elapsed time: {(time.time() - start_time)} secs")

    return user_data_twitter, user_data_ig

In [23]:
user_data_twitter, user_data_ig = get_spi(users_dict["cristiano"], api, cl, retrieve_all_ig=True)

Getting twitter profile data
Elapsed time: 7.618941307067871 secs
Getting instagram profile data
Elapsed time: 360.5560944080353 secs


In [31]:
user_data_twitter

{'user_name': 'Cristiano',
 'n_followers': 105414066,
 'n_retweets': 0,
 'n_tweets': 7,
 'n_retweets_to_user': 189653,
 'n_favorites_to_user': 2314646,
 'n_replies_to_user': 11,
 'created_at': datetime.datetime(2022, 11, 28, 10, 20, 46, 584199),
 'sentiment_twitter': 0.5070352085113999}

In [30]:
user_data_ig

{'user_name': 'cristiano',
 'user_id': '173560420',
 'n_followers': 505143201,
 'n_following': 523,
 'n_posts_total': 3405,
 'n_posts_retrieved': 3398,
 'n_likes_total': 85917263,
 'n_likes_retrieved': 68013684,
 'n_comments_total': 85917263,
 'n_comments_retrieved': 85,
 'created_at': datetime.datetime(2022, 11, 28, 10, 26, 42, 443917),
 'sentiment_instagram': 0.6723071869803803}

In [136]:
start_time_allp = time.time()
users_data = {}

for user in users_dict.keys():
    print(f"Working on {user}")
    user_data_twitter, user_data_ig = get_spi(users_dict[user], api, cl, retrieve_all_ig=False)
    users_data[user] = {}
    users_data[user]["twitter"] = user_data_twitter
    users_data[user]["instagram"] = user_data_ig
    print(f"Done with {user}")
    print("-----------------")
    time.sleep(60)
print(f"Total elapsed time: {(time.time()-start_time_allp)} secs")

Working on cristiano
Getting twitter profile data
Elapsed time: 7.278305292129517 secs
Getting instagram profile data
Elapsed time: 33.23121166229248 secs
Done with cristiano
-----------------
Working on messi
Getting twitter profile data
Elapsed time: 6.320499897003174 secs
Getting instagram profile data
Elapsed time: 34.95846176147461 secs
Done with messi
-----------------
Working on lebron
Getting twitter profile data
Elapsed time: 9.72266960144043 secs
Getting instagram profile data
Elapsed time: 30.344451904296875 secs
Done with lebron
-----------------
Working on nadal
Getting twitter profile data
Elapsed time: 2.567624568939209 secs
Getting instagram profile data
Elapsed time: 38.10001301765442 secs
Done with nadal
-----------------
Working on mayweather
Getting twitter profile data
Elapsed time: 0.7721734046936035 secs
Getting instagram profile data
Elapsed time: 37.77638792991638 secs
Done with mayweather
-----------------
Total elapsed time: 501.10461926460266 secs


## Process the retrieved data and compute spi

### Instagram

In [54]:
users_data_instagram = [users_data[username]["instagram"] for username in users_data.keys()]
df_ig = pd.DataFrame(users_data_instagram)
df_ig["n_likes_retrieved"] = abs(df_ig["n_likes_retrieved"])
df_ig["engagement"] = (df_ig["n_likes_total"] + df_ig["n_comments_total"])/(df_ig["n_followers"])*100
df_ig["relative_engagement"] = (df_ig["n_likes_retrieved"] + df_ig["n_comments_retrieved"])/(df_ig["n_followers"])*100
df_ig


,user_name,user_id,n_followers,n_following,n_posts_total,n_posts_retrieved,n_likes_total,n_likes_retrieved,n_comments_total,n_comments_retrieved,created_at,sentiment_instagram,engagement,relative engagement
0,cristiano,173560420,505143201,523,3405,3400,11098997163,68328814,85948725,71,2022-11-28 12:31:26.591978,0.635150,2214.212894,13.526637
1,leomessi,427553890,380720049,286,950,951,3761903194,64810905,33804075,133,2022-11-28 12:34:07.863611,0.566518,996.981188,17.023279
2,lebron,7855453810,663491,34,2960,2959,18731083,61161,142334,115,2022-11-28 12:38:27.898091,0.638952,2844.562624,9.235393
3,rafaelnadal,1938502255,17103701,211,1248,1247,237967817,1130719,2249351,143,2022-11-28 12:41:16.328042,0.759658,1404.474786,6.611797
4,floydmayweather,16264572,28510733,316,1046,1045,289066065,644987,4563468,137,2022-11-28 12:43:49.077534,0.518799,1029.891210,2.262741


In [129]:
def compute_spi_ig(x):
    spi_ig_weights_dict = {
        "n_followers": 0.1,
        "n_posts_total": 0.1, 
        "n_likes_retrieved": 0.2, 
        "n_comments_retrieved": 0.1, 
        "sentiment_instagram": 0.3, 
        "relative_engagement": 0.2
    }
    spi = 0
    for k in spi_ig_weights_dict.keys():
        spi += spi_ig_weights_dict[k]*x[k]
    return spi

def process_ig_data(data):
    users_data_instagram = [data[username]["instagram"] for username in data.keys()]
    df_ig = pd.DataFrame(users_data_instagram)
    df_ig["relative_engagement"] = np.clip((df_ig["n_likes_retrieved"] + df_ig["n_comments_retrieved"])/(df_ig["n_followers"])*100, 0, 100)
    selected_cols = ["n_followers", "n_posts_total", "n_likes_retrieved", "n_comments_retrieved", "sentiment_instagram", "relative_engagement"]
    df_ig_perc = df_ig.copy()
    for col in selected_cols:
        df_ig_perc[col] = df_ig_perc[col].apply(lambda x: stats.percentileofscore(df_ig_perc[col], x))
    df_ig_perc["spi"] = df_ig_perc[selected_cols].apply(compute_spi_ig, axis=1)
    spi_ig_columns = ["user_name", "n_followers", "n_posts_total", "n_likes_retrieved", "n_comments_retrieved", "sentiment_instagram", "relative_engagement"]
    return pd.concat([df_ig[spi_ig_columns], df_ig_perc["spi"], df_ig["created_at"]], axis=1)

In [134]:
processed_ig_data = process_ig_data(users_data)
processed_ig_data

,user_name,n_followers,n_posts_total,n_likes_retrieved,n_comments_retrieved,sentiment_instagram,relative_engagement,spi,created_at
0,cristiano,505143201,3405,68328814,71,0.635150,13.526637,76.0,2022-11-28 12:31:26.591978
1,leomessi,380720049,950,64810905,133,0.566518,17.023279,64.0,2022-11-28 12:34:07.863611
2,lebron,663491,2960,61161,115,0.638952,9.235393,54.0,2022-11-28 12:38:27.898091
3,rafaelnadal,17103701,1248,1130719,143,0.759658,6.611797,70.0,2022-11-28 12:41:16.328042
4,floydmayweather,28510733,1046,644987,137,0.518799,2.262741,36.0,2022-11-28 12:43:49.077534


In [137]:
processed_ig_data_after = process_ig_data(users_data)
processed_ig_data_after

,user_name,n_followers,n_posts_total,n_likes_retrieved,n_comments_retrieved,sentiment_instagram,relative_engagement,spi,created_at
0,cristiano,505143201,3405,68625403,67,0.629622,13.585350,76.0,2022-11-28 14:56:53.838314
1,leomessi,380720049,950,65051278,141,0.564279,17.086418,58.0,2022-11-28 14:58:29.655310
2,lebron,663491,2960,61729,115,0.639184,9.321001,54.0,2022-11-28 15:00:11.522236
3,rafaelnadal,17103701,1248,1148315,147,0.754209,6.714699,66.0,2022-11-28 15:01:47.364943
4,floydmayweather,28510733,1046,1401874,146,0.566816,4.917517,46.0,2022-11-28 15:03:26.317035


In [139]:
pd_ig = pd.concat([processed_ig_data, processed_ig_data_after], axis=0, ignore_index=True)

#### Workflow:
Each time the whole pipeline starts its execution:

If the user already exists in the database:
1. Extract data
2. Get latest record of user
3. Compute difference in numeric fields
4. Compute relative change
5. Compute spi-bonus
6. Add spi-bonus to base spi
7. Save record to database

If the user does not exist in the database, created and push the data

In [215]:
def compute_bonus_ig_spi(new_data, old_data, username, bonus_weight=0.15):
    pd_ig = pd.concat([old_data, new_data], axis=0, ignore_index=True)
    pd_ig = pd_ig[pd_ig["user_name"] == username].sort_values(by="created_at", ascending=False)[:2].reset_index()
    
    dict_change = {}
    num_cols = [col for col in pd_ig.columns if is_numeric_dtype(pd_ig[col])]
    dict_change["user_name"] = pd_ig["user_name"].unique()[0]
    for col in num_cols:
        dict_change[col] = pd_ig.loc[0][col] - pd_ig.loc[1][col]
    df_change = pd.DataFrame(dict_change, index=[0])
    df_change = df_change.drop("index",axis=1)
    
    dict_relative_change = {}
    dict_change = df_change.to_dict('records')[0]
    dict_base = pd_ig.loc[1].to_dict()

    for key in dict_change.keys():
        if key == "user_name":
            dict_relative_change[key] = dict_change[key]
        elif key == "created_at" or key == "spi":
            pass
        else:
            if dict_base[key] == 0 and dict_change[key] != 0:
                dict_relative_change[key] = 1
            elif dict_base[key] == 0 and dict_change[key] == 0:
                dict_relative_change[key] = 0
            else:
                dict_relative_change[key] = dict_change[key] / dict_base[key]
                
    dict_ig_spi_bonus_weight = {
        'n_followers': 0.1,
        'n_posts_total': 0.1,
        'n_likes_retrieved': 0.1,
        'n_comments_retrieved': 0.1,
        'sentiment_instagram': 0.4,
        'relative_engagement': 0.2
    }

    bonus_spi = 0
    for k in dict_relative_change.keys():
        if k != "user_name":
            bonus_spi += dict_ig_spi_bonus_weight[k]*dict_relative_change[k]

    pd_ig_upd = pd_ig.copy()
    pd_ig_upd.loc[0, "spi"] = pd_ig_upd.loc[0, "spi"] + bonus_weight*bonus_spi*100
    pd_ig_upd = pd_ig_upd.drop("index",axis=1)

    return pd_ig_upd[:1]

In [216]:
df_temp_new_data = pd.DataFrame()
for username in processed_ig_data_after["user_name"].unique():
    df_temp_new_data = pd.concat([df_temp_new_data, compute_bonus_ig_spi(processed_ig_data_after, processed_ig_data, username)], axis=0, ignore_index=True)
df_temp_new_data

,user_name,n_followers,n_posts_total,n_likes_retrieved,n_comments_retrieved,sentiment_instagram,relative_engagement,spi,created_at
0,cristiano,505143201,3405,68625403,67,0.629622,13.585350,75.882808,2022-11-28 14:56:53.838314
1,leomessi,380720049,950,65051278,141,0.564279,17.086418,58.083206,2022-11-28 14:58:29.655310
2,lebron,663491,2960,61729,115,0.639184,9.321001,54.043917,2022-11-28 15:00:11.522236
3,rafaelnadal,17103701,1248,1148315,147,0.754209,6.714699,66.068954,2022-11-28 15:01:47.364943
4,floydmayweather,28510733,1046,1401874,146,0.566816,4.917517,51.933877,2022-11-28 15:03:26.317035


In [217]:
processed_ig_data

,user_name,n_followers,n_posts_total,n_likes_retrieved,n_comments_retrieved,sentiment_instagram,relative_engagement,spi,created_at
0,cristiano,505143201,3405,68328814,71,0.635150,13.526637,76.0,2022-11-28 12:31:26.591978
1,leomessi,380720049,950,64810905,133,0.566518,17.023279,64.0,2022-11-28 12:34:07.863611
2,lebron,663491,2960,61161,115,0.638952,9.235393,54.0,2022-11-28 12:38:27.898091
3,rafaelnadal,17103701,1248,1130719,143,0.759658,6.611797,70.0,2022-11-28 12:41:16.328042
4,floydmayweather,28510733,1046,644987,137,0.518799,2.262741,36.0,2022-11-28 12:43:49.077534


In [218]:
processed_ig_data_after

,user_name,n_followers,n_posts_total,n_likes_retrieved,n_comments_retrieved,sentiment_instagram,relative_engagement,spi,created_at
0,cristiano,505143201,3405,68625403,67,0.629622,13.585350,76.0,2022-11-28 14:56:53.838314
1,leomessi,380720049,950,65051278,141,0.564279,17.086418,58.0,2022-11-28 14:58:29.655310
2,lebron,663491,2960,61729,115,0.639184,9.321001,54.0,2022-11-28 15:00:11.522236
3,rafaelnadal,17103701,1248,1148315,147,0.754209,6.714699,66.0,2022-11-28 15:01:47.364943
4,floydmayweather,28510733,1046,1401874,146,0.566816,4.917517,46.0,2022-11-28 15:03:26.317035


### Twitter

In [125]:
def compute_spi_tw(x):
    spi_ig_weights_dict = {
        "n_followers": 0.1,
        "n_retweets": 0.1, 
        "n_tweets": 0.1, 
        "n_retweets_to_user": 0.1, 
        "n_favorites_to_user": 0.1, 
        "n_replies_to_user": 0.1,
        "sentiment_twitter": 0.3,
        "relative_engagement": 0.1
    }
    spi = 0
    for k in spi_ig_weights_dict.keys():
        spi += spi_ig_weights_dict[k]*x[k]
    return spi

def process_tw_data(data):
    users_data_twitter = [data[username]["twitter"] for username in data.keys()]
    df_tw = pd.DataFrame(users_data_twitter)
    df_tw["relative_engagement"] = (df_tw.apply(lambda x: (x["n_favorites_to_user"] + x["n_retweets_to_user"] + x["n_replies_to_user"])/(x["n_followers"])*100 if x["n_tweets"] > 0 else 0, axis=1))
    selected_cols = ["n_followers", "n_retweets", "n_tweets", "n_retweets_to_user", "n_favorites_to_user", "n_replies_to_user", "sentiment_twitter", "relative_engagement"]
    df_tw_perc = df_tw.copy()
    for col in selected_cols:
        df_tw_perc[col] = df_tw_perc[col].apply(lambda x: stats.percentileofscore(df_tw_perc[col], x))
    df_tw_perc["spi"] = df_tw_perc[selected_cols].apply(compute_spi_tw, axis=1)
    spi_tw_columns = ["user_name","n_followers","n_retweets","n_tweets","n_retweets_to_user","n_favorites_to_user","n_replies_to_user","sentiment_twitter","relative_engagement"]
    return pd.concat([df_tw[spi_tw_columns], df_tw_perc["spi"], df_tw["created_at"]], axis=1)

In [133]:
processed_tw_data = process_tw_data(users_data)
processed_tw_data

,user_name,n_followers,n_retweets,n_tweets,n_retweets_to_user,n_favorites_to_user,n_replies_to_user,sentiment_twitter,relative_engagement,spi,created_at
0,Cristiano,105419530,0,7,188104,2330900,13,0.523705,2.389516,91.0,2022-11-28 12:27:58.053931
1,Ieomessiok,588991,0,5,25194,338329,10,0.489405,61.721317,51.0,2022-11-28 12:32:41.526963
2,KingJames,52481548,1,9,8084,170410,11,0.518708,0.340129,78.0,2022-11-28 12:35:26.502346
3,RafaelNadal,15823763,0,1,181,1676,0,0.500000,0.011736,45.0,2022-11-28 12:39:39.046504
4,FloydMayweather,7830391,0,0,0,0,0,0.500000,0.000000,35.0,2022-11-28 12:42:28.621171


In [138]:
processed_tw_data_after = process_tw_data(users_data)
processed_tw_data_after

,user_name,n_followers,n_retweets,n_tweets,n_retweets_to_user,n_favorites_to_user,n_replies_to_user,sentiment_twitter,relative_engagement,spi,created_at
0,Cristiano,105422265,0,7,184446,2341677,3,0.502727,2.396198,81.0,2022-11-28 14:56:25.676973
1,Ieomessiok,589868,0,5,25217,338935,10,0.631121,61.736185,77.0,2022-11-28 14:58:04.245619
2,KingJames,52480086,1,9,8537,181802,14,0.423578,0.362715,62.0,2022-11-28 14:59:48.942376
3,RafaelNadal,15823636,0,1,181,1678,0,0.500000,0.011748,45.0,2022-11-28 15:01:23.024564
4,FloydMayweather,7830645,0,0,0,0,0,0.500000,0.000000,35.0,2022-11-28 15:03:01.904700


In [232]:
def compute_bonus_tw_spi(new_data, old_data, username, bonus_weight=0.15):
    pd_tw = pd.concat([old_data, new_data], axis=0, ignore_index=True)
    pd_tw = pd_tw[pd_tw["user_name"] == username].sort_values(by="created_at", ascending=False)[:2].reset_index()
    
    dict_change = {}
    num_cols = [col for col in pd_tw.columns if is_numeric_dtype(pd_tw[col])]
    dict_change["user_name"] = pd_tw["user_name"].unique()[0]
    for col in num_cols:
        dict_change[col] = pd_tw.loc[0][col] - pd_tw.loc[1][col]
    df_change = pd.DataFrame(dict_change, index=[0])
    df_change = df_change.drop("index",axis=1)
    
    dict_relative_change = {}
    dict_change = df_change.to_dict('records')[0]
    dict_base = pd_tw.loc[1].to_dict()

    for key in dict_change.keys():
        if key == "user_name":
            dict_relative_change[key] = dict_change[key]
        elif key == "created_at" or key == "spi":
            pass
        else:
            if dict_base[key] == 0 and dict_change[key] != 0:
                dict_relative_change[key] = 1
            elif dict_base[key] == 0 and dict_change[key] == 0:
                dict_relative_change[key] = 0
            else:
                dict_relative_change[key] = dict_change[key] / dict_base[key]
    dict_tw_spi_bonus_weight = {
        'n_followers': 0.1,
        'n_retweets': 0.1,
        'n_tweets': 0.1,
        'n_retweets_to_user': 0.1,
        'n_favorites_to_user': 0.1,
        'n_replies_to_user': 0.1,
        'sentiment_twitter': 0.3,
        'relative_engagement': 0.1
    }

    bonus_spi = 0
    for k in dict_relative_change.keys():
        if k != "user_name":
            bonus_spi += dict_tw_spi_bonus_weight[k]*dict_relative_change[k]

    pd_tw_upd = pd_tw.copy()
    pd_tw_upd.loc[0, "spi"] = pd_tw_upd.loc[0, "spi"] + bonus_weight*bonus_spi*100
    pd_tw_upd = pd_tw_upd.drop("index",axis=1)

    return pd_tw_upd[:1]

In [233]:
df_temp_new_data = pd.DataFrame()
for username in processed_tw_data_after["user_name"].unique():
    df_temp_new_data = pd.concat([df_temp_new_data, compute_bonus_tw_spi(processed_tw_data_after, processed_tw_data, username)], axis=0, ignore_index=True)
df_temp_new_data

,user_name,n_followers,n_retweets,n_tweets,n_retweets_to_user,n_favorites_to_user,n_replies_to_user,sentiment_twitter,relative_engagement,spi,created_at
0,Cristiano,105422265,0,7,184446,2341677,3,0.502727,2.396198,79.647898,2022-11-28 14:56:25.676973
1,Ieomessiok,589868,0,5,25217,338935,10,0.631121,61.736185,78.309713,2022-11-28 14:58:04.245619
2,KingJames,52480086,1,9,8537,181802,14,0.423578,0.362715,61.867694,2022-11-28 14:59:48.942376
3,RafaelNadal,15823636,0,1,181,1678,0,0.500000,0.011748,45.003405,2022-11-28 15:01:23.024564
4,FloydMayweather,7830645,0,0,0,0,0,0.500000,0.000000,35.000049,2022-11-28 15:03:01.904700


In [234]:
processed_tw_data_after

,user_name,n_followers,n_retweets,n_tweets,n_retweets_to_user,n_favorites_to_user,n_replies_to_user,sentiment_twitter,relative_engagement,spi,created_at
0,Cristiano,105422265,0,7,184446,2341677,3,0.502727,2.396198,81.0,2022-11-28 14:56:25.676973
1,Ieomessiok,589868,0,5,25217,338935,10,0.631121,61.736185,77.0,2022-11-28 14:58:04.245619
2,KingJames,52480086,1,9,8537,181802,14,0.423578,0.362715,62.0,2022-11-28 14:59:48.942376
3,RafaelNadal,15823636,0,1,181,1678,0,0.500000,0.011748,45.0,2022-11-28 15:01:23.024564
4,FloydMayweather,7830645,0,0,0,0,0,0.500000,0.000000,35.0,2022-11-28 15:03:01.904700


In [235]:
processed_tw_data

,user_name,n_followers,n_retweets,n_tweets,n_retweets_to_user,n_favorites_to_user,n_replies_to_user,sentiment_twitter,relative_engagement,spi,created_at
0,Cristiano,105419530,0,7,188104,2330900,13,0.523705,2.389516,91.0,2022-11-28 12:27:58.053931
1,Ieomessiok,588991,0,5,25194,338329,10,0.489405,61.721317,51.0,2022-11-28 12:32:41.526963
2,KingJames,52481548,1,9,8084,170410,11,0.518708,0.340129,78.0,2022-11-28 12:35:26.502346
3,RafaelNadal,15823763,0,1,181,1676,0,0.500000,0.011736,45.0,2022-11-28 12:39:39.046504
4,FloydMayweather,7830391,0,0,0,0,0,0.500000,0.000000,35.0,2022-11-28 12:42:28.621171


### Database connection

In [237]:
# Module Imports
import mariadb
import sys

# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="root",
        password="itsense",
        host="localhost",
        port=3306,
        database="vertikal"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [243]:
users_dict = {
    "cristiano": {
        "instagram": "cristiano",
        "twitter": "Cristiano"
    },
    "messi": {
        "instagram": "leomessi",
        "twitter": "Ieomessiok"
    },
    "lebron": {
        "instagram": "lebron",
        "twitter": "KingJames"
    },
    "nadal": {
        "instagram": "rafaelnadal",
        "twitter": "RafaelNadal"
    },
    "mayweather": {
        "instagram": "floydmayweather",
        "twitter": "FloydMayweather"
    }
}

In [245]:
for username in users_dict.keys():
    ig_username = users_dict[username]["instagram"]
    tw_username = users_dict[username]["twitter"]
    str_to_ex = f"INSERT INTO users (name, instagram_username, twitter_username, last_update) VALUES ('{username}', '{ig_username}', '{tw_username}', now());"
    cur.execute(str_to_ex)
conn.commit()

In [253]:
cur.execute("SELECT * FROM users;") 

for id, name, instagram_username, twitter_username, last_update in cur: 
    print(f"name: {name}, instagram_username: {instagram_username}, twitter_username: {twitter_username}, last_update: {last_update}")

name: cristiano, instagram_username: cristiano, twitter_username: Cristiano, last_update: 2022-11-28 17:07:35
name: messi, instagram_username: leomessi, twitter_username: Ieomessiok, last_update: 2022-11-28 17:07:35
name: lebron, instagram_username: lebron, twitter_username: KingJames, last_update: 2022-11-28 17:07:35
name: nadal, instagram_username: rafaelnadal, twitter_username: RafaelNadal, last_update: 2022-11-28 17:07:35
name: mayweather, instagram_username: floydmayweather, twitter_username: FloydMayweather, last_update: 2022-11-28 17:07:35


In [ ]:
str_to_ex = f"INSERT INTO users (name, instagram_username, last_update) VALUES ('FalcaoGarcia', 'falcao', now());"
cur.execute(str_to_ex)
conn.commit()

In [ ]:
conn.close()

---

---